In [26]:
from axikernels.core.handlers.element_output import ElementOutput
import matplotlib.pyplot as plt

In [27]:
a = ElementOutput('data/EXAMPLE_FORWARD_iso/output/elements')
b = ElementOutput('data/EXAMPLE_3D_element/output/elements')

In [28]:
points = [[6371000, 0, 50]]

In [32]:
A = a.load_data(points=points, channels=['UZ'])
B = b.load_data(points=points, channels=['UZ'])
plt.plot(A[0][0])
plt.plot(B[0][0])
plt.xlim([400, 600])
plt.show()

QObject: Cannot create children for a parent that is in a different thread.
(Parent is QApplication(0x7f2f0db77660), parent's thread is QThread(0x7f2f0db6e3c0), current thread is QThread(0x559b89ebb030)
Loading and interpolating: 100%|██████████| 1/1 [00:00<00:00, 503.76point/s]


: 